## Logging on

Use Selenium to visit https://webapps1.chicago.gov/buildingrecords/ and accept the agreement.

> Think about when you use `.find_element...` and when you use `.find_elementSSS...`

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from webdriver_manager.firefox import GeckoDriverManager

/Users/shreyaraman/.pyenv/versions/3.10.3/lib/python3.10/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn(
/var/folders/71/p219_qns5mv0bqxkh541lpl40000gn/T/ipykernel_66508/1912161385.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())


In [2]:
driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())

/var/folders/71/p219_qns5mv0bqxkh541lpl40000gn/T/ipykernel_66508/3328504527.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())


In [3]:
driver.get("https://webapps1.chicago.gov/buildingrecords/")

In [4]:
driver.find_element(By.ID, "rbnAgreement1").click()

In [5]:
driver.find_element(By.ID, "submit").click()

## Searching

Search for **400 E 41ST ST**.

In [6]:
driver.find_element(By.ID, "fullAddress").send_keys("400 E 41ST ST")

In [7]:
driver.find_element(By.ID, "submit").click()

## Saving tables with pandas

Use pandas to save a CSV of all **permits** to `Permits - 400 E 41ST ST.csv`. Note that there are **different sections of the page**, not just one long permits table.

> - *Tip: When using `.read_html`, try using `flavor='lxml'` and comparing the results to `flavor='html5lib'`. Which works better?*
> - *Tip: You might need to install `html5lib` using `pip`. If so, you'll need to restart the notebook using **Kernel > Restart** before it will work.*

In [16]:
df=pd.read_html(driver.page_source, flavor="html5lib")[0]

In [17]:
df.to_csv("Permits - 400 E 41ST ST.csv", index=False)

## Saving tables the long way

Save a CSV of all DOB inspections to `Inspections - 400 E 41ST ST.csv`.

This is more complicated than the last one becuse **we also need to save the URL to the inspection** (see how the inspection number is a link?). As a result, you won't be able to use pandas! Instead, you'll need to use a loop and create a list of dictionaries.

You can use Selenium or you can feed the source to BeautifulSoup. You should have approximately 160 rows.

You'll probably need to find the table first, then the rows inside, then the cells inside of each row. You'll probably use lots of list indexing. I might recommend XPath for finding the table.

*Tip: If you get a "list index out of range" error, it's probably due to an issue involving `thead` vs `tbody` elements. What are they? What are they for? What's in them? There are a few ways to troubleshoot it.*

In [44]:
df=pd.read_html(driver.page_source, flavor="html5lib")[2]

In [45]:
doc=BeautifulSoup(driver.page_source)

In [46]:
urls=[]
table=doc.find_all("table", id="resultstable_inspections")

for data in table:
    for links in data.select("a"):
        link=links['href']
        
        urls.append(link)

In [47]:
df['LINKS']=urls

In [51]:
df.head()

,INSP #,INSPECTION DATE,STATUS,TYPE DESCRIPTION,LINKS
0,13821288,05/16/2022,PASSED,PERMIT INSPECTION,/buildingrecords/inspectiondetails?addr=364923...
1,13813188,05/16/2022,PASSED,ANNUAL INSPECTION,/buildingrecords/inspectiondetails?addr=364923...
2,13808208,05/13/2022,FAILED,PERMIT INSPECTION,/buildingrecords/inspectiondetails?addr=364923...
3,13748085,04/25/2022,FAILED,ANNUAL INSPECTION,/buildingrecords/inspectiondetails?addr=364923...
4,13684797,03/21/2022,FAILED,ANNUAL INSPECTION,/buildingrecords/inspectiondetails?addr=364923...


### Bonus preview of Wednesday's content

**You prrrrrobably shouldn't do this one unless you want a real challenge.**

If you click the inspection number, it'll open up a new window that shows you details of the violations from that visit. Count the number of violations for each visit and save it in a new column called **num_violations**.

Save this file as `Inspections - 400 E 41ST ST - with counts.csv`.

Since clicking the link opens in a new window, we have to say "Hey Selenium, pay attention to that new window!" We do that with `driver.switch_to.window(driver.window_handles[-1])` (each window gets a `window_handle`, and we're just asking the driver to switch to the last one.). A rough sketch of what your code will look like is here:

```python
# Click the link that opens the new window

# Switch to the new window/tab
driver.switch_to.window(driver.window_handles[-1])

# Do your scraping in here

# Close the new window/tab
driver.close()

# Switch back to the original window/tab
driver.switch_to.window(driver.window_handles[0])
```

You'll want to play around with them individually before you try it with the whole set - the ones that pass are very different pages than the ones with violations! There are a few ways to get the number of violations, some easier than others.

In [56]:
driver.find_element(By.XPATH, "/html/body/div/div[4]/div[10]/div/table/tbody/tr[3]/td[1]/a").click()

In [57]:
driver.switch_to.window(driver.window_handles[-1])

In [59]:
driver.find_element(By.ID, "resultstable_info").text

'Showing 1 to 1 of 1 entries'

In [60]:
driver.close()

In [61]:
driver.switch_to.window(driver.window_handles[0])

In [62]:
driver.find_element(By.XPATH, "/html/body/div/div[4]/div[10]/div/table/tbody/tr[1]/td[1]/a").click()

In [65]:
driver.switch_to.window(driver.window_handles[-1])

In [66]:
driver.find_element(By.TAG_NAME, "h5").text

'There are no Alleged Code Violation records for Inspection 13821288'

I think this is where I get stuck. Ideally I would want to do if and elif. Something like,

if the status column says "FAILED", do 

```python
driver.find_element(By.XPATH, "/html/body/div/div[4]/div[10]/div/table/tbody/tr[1]/td[1]/a").click()

driver.switch_to.window(driver.window_handles[-1])

driver.find_element(By.ID, "resultstable_info").text

driver.close()

driver.switch_to.window(driver.window_handles[0])

```

else do

```python
driver.find_element(By.XPATH, "/html/body/div/div[4]/div[10]/div/table/tbody/tr[3]/td[1]/a").click()

driver.switch_to.window(driver.window_handles[-1])

driver.find_element(By.TAG_NAME, "h5").text

driver.close()

driver.switch_to.window(driver.window_handles[0])

```
